<a href="https://colab.research.google.com/github/jessijzhao/FedCD/blob/master/FedCD_Federated_Cloning_and_Deletion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **FedCD: Federated Cloning and Deletion**

Jessica Zhao, Eric Lin, Kavya Kopparapu 

Presented at the 3rd International Workshop on Artificial Intelligence of Things
in conjunction with the 26th ACM SIGKDD International Conference on Knowledge Discovery and Data Mining (KDD 2020).

Utilizes handout code for federated learning with CIFAR-10 from CS 242: Computing at Scale (Spring 2020) by HT Kung.






### **Parameters**

In [1]:
# setup parameters
NUM_DEVICES = 30                                            # number of devices
NUM_LABELS = 10                                             # number of labels
NUM_TRAIN_PER_DEVICE = 5000                                 # number of training examples per device
NUM_VALID_PER_DEVICE = NUM_TRAIN_PER_DEVICE // 3            # number of validation examples per device
NUM_TEST_PER_DEVICE = 500                                   # number of test examples per device
DISTRIBUTION = "uniform"                                    # data distribution
assert (DISTRIBUTION in ["uniform", "hypergeometric"])

# training parameters
NUM_ROUNDS = 150                                            # number of training rounds
NUM_LOCAL_EPOCHS = 3                                        # number of local epochs per device per round
DEVICE_PCT = 0.5                                            # fraction of devices for each round
DUPLICATE_MILESTONES = [2, 15, 25, 30]                      # training rounds at which to duplicate models, TODO reset
THRESHOLD = 15                                              # number of rounds before finalizing a model

# quantization parameters
QUANTIZE = True                                             # whether to quantize models
NBIT = 8                                                    # number of bits to quantize to

# which model(s) to run
RUN_FEDCD = True                                            # Federated Cloning and Deletion
RUN_FEDAVG = True                                           # baseline federated learning algorithm

### **Setup Code**


In [2]:
import time
import copy
import random
from collections import OrderedDict

import torch
import torch.nn as nn
from torch import optim
import numpy as np


class DatasetSplit(torch.utils.data.Dataset):
    """
    Returns a new dataset with the indexed items.
    """
    def __init__(self, dataset, idxs):
        self.dataset = dataset
        self.idxs = [int(i) for i in idxs]

    def __len__(self):
        return len(self.idxs)

    def __getitem__(self, item):
        image, label = self.dataset[self.idxs[item]]
        return image, torch.tensor(label)

In [3]:
def conv_block(in_channels, out_channels, kernel_size=3, stride=1,
               padding=1):
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding,
                  bias=False),
        nn.BatchNorm2d(out_channels),
        nn.ReLU(inplace=True)
    )
    
class ConvNet(nn.Module):
    """
    Convolution neural net used as the model.
    """
    def __init__(self):
        super(ConvNet, self).__init__()
        self.model = nn.Sequential(
            conv_block(3, 32),
            conv_block(32, 32),
            conv_block(32, 64, stride=2),
            conv_block(64, 64),
            conv_block(64, 64),
            conv_block(64, 128, stride=2),
            conv_block(128, 128),
            conv_block(128, 256),
            conv_block(256, 256),
            nn.AdaptiveAvgPool2d(1)
            )

        self.classifier = nn.Linear(256, 10)

    def forward(self, x):
        h = self.model(x)
        B, C, _, _ = h.shape
        h = h.view(B, C)
        return self.classifier(h)

In [4]:
import statistics 

class Device():
    """
    Each instance represents an edge device.
    """
    def __init__(self, net, device_id, trainset, validset, testset, train_idxs, 
                 valid_idxs, test_idxs, bias, archetype, lr=0.1, milestones=[25, 50, 75], 
                 batch_size=128):

        # training data
        self.device_trainset = DatasetSplit(trainset, train_idxs)
        self.trainloader = torch.utils.data.DataLoader(self.device_trainset,
                                                        batch_size=batch_size,
                                                        shuffle=True,
                                                        num_workers=2)
        # validation data
        self.device_validset = DatasetSplit(validset, valid_idxs)
        self.validloader = torch.utils.data.DataLoader(self.device_validset,
                                                        batch_size=batch_size,
                                                        shuffle=True,
                                                        num_workers=2)
        # test data
        self.device_testset = DatasetSplit(testset, test_idxs)
        self.testloader = torch.utils.data.DataLoader(self.device_testset,
                                                        batch_size=batch_size,
                                                        shuffle=True,
                                                        num_workers=2)
        
        # initiate list of models with a single model
        device_net = copy.deepcopy(net)
        optimizer = torch.optim.SGD(device_net.parameters(), lr=lr, momentum=0.9,
                                    weight_decay=5e-4)
        scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer,
                                                         milestones=milestones,
                                                         gamma=0.1)
        self.nets = [{
            'net': device_net,
            'optimizer': optimizer,
            'scheduler': scheduler,
            'train_loss_tracker': [],
            'train_acc_tracker': [],
            'valid_loss_tracker': [],
            'valid_acc_tracker': [],
            'test_loss_tracker': [],
            'test_acc_tracker': [],
        }]
        
        # device id
        self.idx = device_id

        # rankings for each model, must sum to 1
        self.ranking = [1.]

        # whether a given model is active, in (1, 0)
        self.active = [1] 

        # bias and archetype parameters
        self.bias = bias              # fractions to represent a linear combination of archetypes
        self.archetype = archetype    # list of possible archetypes
    
    def update_ranking(self, prune, removed=False, duplicate_model_id=-1, offset_rank=-1):
        # number of standard deviations away for model deletion cutoff
        zero_threshold = 1 

        if len(self.nets) > 1:
            metrics = []
            for i in range(len(self.nets)):
                if len(self.nets[i]['valid_acc_tracker']) > 0:
                    rank = self.nets[i]['valid_acc_tracker'][-1]
                    if len(self.nets[i]['valid_acc_tracker']) >= 3:
                        rank = ((self.nets[i]['valid_acc_tracker'][-1]
                                 + self.nets[i]['valid_acc_tracker'][-2]
                                 + self.nets[i]['valid_acc_tracker'][-3]) /3)
                    if duplicate_model_id == i and offset_rank != -1:
                        # Heavily rank the devices that are underperforming for new models and vice versa
                        rank = offset_rank      
                    if rank == 0:
                        rank += 0.001
                    metrics.append(rank)
                else:
                    metrics.append(50)
            
            # if we added more models, add active trackers for them
            while len(self.nets) != len(self.active):
                self.active.append(1)
            #  Auto-set a model as inactive if it was already removed
            if removed:       
                self.active[duplicate_model_id] = 0

            # normalization first time (with self.active)
            self.ranking = [metrics[i]*self.active[i]/sum(metrics) for i in range(len(metrics))]

            nonzero_elts = np.array(self.active).nonzero()[0]
            nonzero_arr = []
            for i in nonzero_elts:
                nonzero_arr.append(self.ranking[i])
            
            # Remove models that are underperforming
            if offset_rank == -1:   # only remove if not duplicating round
                max_rank = max(self.ranking)
                if len(nonzero_elts) > 1:
                    std = statistics.stdev(nonzero_arr)
                    mean = sum(nonzero_arr)/len(nonzero_arr)

                    # remove models that are underperforming
                    for j in range(len(self.ranking)):
                        if self.active[j] != 0:
                            if mean - self.ranking[j] > zero_threshold*std :
                                self.ranking[j], self.active[j] = 0, 0
                            elif len(self.ranking) > 3 and (self.ranking[j] * 10 < max_rank):
                                self.ranking[j], self.active[j] = 0, 0

            bool_ranking_below_zero = False
            
            # Add noise
            noise = random.gauss(0, 0.01)
            if len(nonzero_elts) == 1:
                i = nonzero_elts[0]
            else:
                i = nonzero_elts[random.randint(0, len(nonzero_elts)-1)]
                for j in range(len(self.ranking)):
                    if j != i:
                        self.ranking[j] -= noise/(len(nonzero_elts)-1)
                        if self.ranking[j] < 0:
                          bool_ranking_below_zero = True
            
            self.ranking[i] += noise
            if self.ranking[i] < 0:
                bool_ranking_below_zero = True

            # normalize again
            if bool_ranking_below_zero:
                self.ranking = [self.ranking[i]-min(self.ranking) for i in range(len(self.ranking))]
            self.ranking = [self.ranking[i]*self.active[i]/sum(self.ranking) for i in range(len(self.ranking))]

            # finalize a model if applicable s
            if prune and sum(self.active) == 2:
                m1, m2 = [i for i in range(len(self.active)) if self.active[i] == 1 ]
                # ensure that m1 is the better model
                if self.ranking[m2] > self.ranking[m1]:
                    m1, m2 = m2, m1
                # if m1 is significantly better, deactivate m2
                if self.ranking[m1] - self.ranking[m2] > 0.5:
                    self.ranking[m1] = 1
                    self.ranking[m2], self.active[m2] = 0, 0
                    

In [5]:
class Analytics():
    """
    Tracks and analyzes experimental data
    """
    def __init__(self):
        self.per_device_accuracy = []

    def update_per_device(self, per_device):
        self.per_device_accuracy.append(per_device)

In [6]:
def train(epoch, device, model_id):
    device.nets[model_id]['net'].train()
    train_loss, correct, total = 0, 0, 0

    dataset = device.device_trainset
    dataloader = device.trainloader

    for batch_idx, (inputs, targets) in enumerate(dataloader):
        inputs, targets = inputs.cuda(), targets.cuda()
        device.nets[model_id]['optimizer'].zero_grad()
        outputs = device.nets[model_id]['net'](inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        device.nets[model_id]['optimizer'].step()
        train_loss += loss.item()
        device.nets[model_id]['train_loss_tracker'].append(loss.item())
        loss = train_loss / (batch_idx + 1)
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        acc = 100. * correct / total
        dev_id = device.idx
    test_loss = 0
    outputs = [0]
    device.nets[model_id]['train_acc_tracker'].append(acc)

def validate(epoch, device, model_id):
    device.nets[model_id]['net'].eval()
    test_loss, correct, total = 0, 0, 0

    dataset = device.device_validset
    dataloader = device.validloader
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(dataloader):
            inputs, targets = inputs.cuda(), targets.cuda()
            outputs = device.nets[model_id]['net'](inputs)
            loss = criterion(outputs, targets)
            test_loss += loss.item()
            device.nets[model_id]['valid_loss_tracker'].append(loss.item())
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
            loss = test_loss / (batch_idx + 1)
            acc = 100.* correct / total
        test_loss = 0
        outputs = [0]
    acc = 100.*correct/total
    device.nets[model_id]['valid_acc_tracker'].append(acc)
    device.nets[model_id]['net'].train()

def test(epoch, device, model_id, dataset, dataloader=None, label_dict=None):
    """
    Uses test set to evaluate performance.
    """
    # turn the net into evaluaton mode
    device.nets[model_id]['net'].eval()

    criterion = nn.CrossEntropyLoss()
    test_loss, correct, total = 0, 0, 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(dataloader):
            inputs, targets = inputs.cuda(), targets.cuda()
            outputs = device.nets[model_id]['net'](inputs)
            loss = criterion(outputs, targets)
            test_loss += loss.item()            
            device.nets[model_id]['test_loss_tracker'].append(loss.item())
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
            loss = test_loss / (batch_idx + 1)
            acc = 100.* correct / total
        test_loss = 0
        outputs = [0]
    acc = 100.*correct/total
    device.nets[model_id]['test_acc_tracker'].append(acc)

    device.nets[model_id]['net'].train()
    return ('%.3f' % loss, '%.3f' % acc)

In [7]:
def model_average_weight(devices, model_id):
    '''
    devices: list of devices
    model_id: id of given model
    
    Returns the average of the weights of the given model taken over devices.
    '''
    d_id = 0
    while d_id < len(devices) and devices[d_id].active[model_id] == 0:
        d_id += 1

    if d_id >= len(devices):
        return None

    # initialize a global tensor with the weights of the first device   
    global_tensors = copy.deepcopy(devices[d_id].nets[model_id]['net'].state_dict()) 
    ranking_sum = devices[d_id].ranking[model_id]

    #iterate over the remaining devices
    for i in range(len(devices)):
        if i == d_id:
            #add the tensors together by the key they are indexed by
            for j in global_tensors.keys(): 
                global_tensors[j] = global_tensors[j] * devices[d_id].ranking[model_id]
        if devices[i].active[model_id] == 1:
            # store the device and the state_dict for easier referencing
            d = devices[i]
            d_tensors = d.nets[model_id]['net'].state_dict()
            # add the tensors together by the key they are indexed by
            for j in global_tensors.keys(): 
                global_tensors[j] += (d_tensors[j] * d.ranking[model_id]).type_as(global_tensors[j])
            ranking_sum += devices[i].ranking[model_id]

    # average each tensor by the number of devices
    for j in global_tensors.keys(): 
        global_tensors[j] = global_tensors[j]/float(ranking_sum)

    return global_tensors

#### **Create non-iid training and validation sets**

In [8]:
# creates noniid TEST datasets for each group
def gen_label_dict(dataset):
    '''
    dataset: PyTorch Dataset (e.g., CIFAR-10 training set)

    Returns a dictionary of the following format:
      {
        0: [3, 65, 2233, ..., 22]
        1: [0, 2, 4, ..., 583]
        ...
      }
    '''
    # label dict stores the indexes of the dataset examples that fall into the ith group of CIFAR
    label_dict = {}

    # assumes CIFAR with labels 0-9
    for i in range(10): 
        label_dict[i] = []
    for i in range(len(dataset)):
        label = dataset[i][1]
        label_dict[label].append(i)
    return label_dict

# creates noniid TRAINING and VALIDATION datasets for each group
def data_sampler(dist, dataset, num_items_per_device, archetype, bias):
    '''
    dataset: PyTorch Dataset (e.g., CIFAR-10 training set)
    num_items_per_device: how many samples to assign to each device
    archetype: a dictionary of arrays representing the labels that is predominantly represented by this edge device
        device index -> array of archetypes
    bias: a dictionary of the percent of samples that are represented by the archetype
        device index -> value from 0 to 1

    Returns a dictionary of the following format:
      {
        0: [3, 65, 2233, ..., 22] // device 0 sample indexes
        1: [0, 2, 4, ..., 583] // device 1 sample indexes
        ...
      }
    '''
    assert (dist in ["uniform", "hypergeometric"])

    label_dict = gen_label_dict(dataset)
    result_dict = {}

    if dist == "uniform":
        for i in range(NUM_DEVICES):
            bias_group = []
            not_bias_group = []
            archs = range(10)
            for j in label_dict.keys():
                if j in archetype[i]:
                    bias_group += label_dict[j]
                # two meta-archetypes
                if archetype[i][0] in range(5): 
                    if j in range(5) and j != archetype[i][0]:
                        not_bias_group += label_dict[j]
                # two meta-archetypes
                elif archetype[i][0] in range(5,10): 
                    if j in range(5,10) and j != archetype[i][0]:
                        not_bias_group += label_dict[j]

            exs = random.sample(bias_group, int(num_items_per_device*bias[i]))
            exs.extend(random.sample(not_bias_group, num_items_per_device-int(num_items_per_device*bias[i])))
            random.shuffle(exs)
            result_dict[i] = exs
        return result_dict

    if dist == "hypergeometric":
        rng = np.random.default_rng()
        
        for i in range(NUM_DEVICES):
            N = 110
            ngood = archetype[i][0]*20 + 5
            nbad = N - ngood
            nsamp = 10
            classes = rng.hypergeometric(ngood, nbad, nsamp, num_items_per_device)
            unique_elements, counts_elements = np.unique(classes, return_counts=True)

            # how many examples from each label
            counts = dict(zip(list(unique_elements), list(counts_elements)))
            for j in range(10):
                if j not in counts:
                    counts[j] = 0

            exs = []
            for j in label_dict.keys():
                exs.extend(random.sample(label_dict[j], counts[j]))

            random.shuffle(exs)
            result_dict[i] = exs
        return result_dict

#### **Quantization**

In [9]:
def quantizer(input, nbit):
    '''    
    input: a full precision tensor
    nbit: the number of bits in the quantized representation
    
    Returns a quantized nbit version of a full precision tensor.
    '''
    scale_factor = 1 / (2**nbit -  1)

    # scale input by inverse of scale_factor and round to nearest integer
    output = input / scale_factor
    output = torch.round(output)

    # scale rounded output back and return
    output *= scale_factor
    return output


def quantize_model(model, nbit):
    '''
    model: ConvNet model
    nbit: the number of bits in the quantized representation

    Quantizes the ConvNet model.
    '''
    for m in model.modules():
        if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
            m.weight.data, m.adaptive_scale = dorefa_g(m.weight, nbit)
            if m.bias is not None:
                m.bias.data,_ = dorefa_g(m.bias, nbit, m.adaptive_scale)


def dorefa_g(w, nbit, adaptive_scale=None):
    '''
    w: a floating-point weight tensor to quantize
    nbit: the number of bits in the quantized representation
    adaptive_scale: the maximum scale value. if None, it is set to be the
                    absolute maximum value in w.
    '''
    if adaptive_scale is None:
        adaptive_scale = torch.max(torch.abs(w))
    
    sigma = torch.rand(w.shape) - 0.5
    noise = sigma / (2**nbit - 1)

    # avoid type errors
    noise = noise.type(w.type())
    inp = w / (2*adaptive_scale) + 0.5 + noise
    w_q = 2*adaptive_scale * (quantizer(inp, nbit) - 0.5)

    return w_q, adaptive_scale

#### **Federated Learning Algorithm**

In [10]:
def federated_learning(algorithm="FedAvg"):
    """
    Runs federated learning algorithm (either FedCD or FedAvg).
    """
    assert (algorithm in ["FedCD", "FedAvg"])

    model_id_lst = [0]
    prune = False
    tracker = Analytics()

    # create devices
    devices = [Device(net, i, trainset, validset, testset, train_idxs[i], valid_idxs[i], 
                      test_idxs_device[i], devices_bias[i], devices_archetype[i]) 
               for i in range(NUM_DEVICES)]
    print('Devices', len(devices))

    start_time = time.time()
    for round_num in range(NUM_ROUNDS):

        if round_num >= THRESHOLD:
            prune = True
      
        # select a percentage of devices to participate in each training round
        round_devices = random.sample(devices, k=round(DEVICE_PCT*len(devices)))

        for device in round_devices:
            for model_id in model_id_lst:
                if device.active[model_id] != 0:
                    # Training
                    for local_epoch in range(NUM_LOCAL_EPOCHS):
                        train(local_epoch, device, model_id) 
                        # print("Device: " + str(device.idx) + " VALIDATION: model_id # " + str(model_id))
                        # validate(local_epoch, device, model_id) 
                        # print()
                    # after training, quantize the learned model
                    if QUANTIZE:
                        quantize_model(device.nets[model_id]['net'], NBIT)
            
        for model_id in model_id_lst:
            w_avg = model_average_weight(round_devices, model_id)

            if w_avg != None:
                for device in devices:
                    if device.active[model_id]!= 0:
                        device.nets[model_id]['net'].load_state_dict(w_avg)
                        device.nets[model_id]['optimizer'].zero_grad()
                        device.nets[model_id]['optimizer'].step()
                        device.nets[model_id]['scheduler'].step()
                
            # test accuracy with highest ranking model
            if devices[0].active[model_id] == 1 and devices[0].ranking[model_id] == max(devices[0].ranking):
                # print()
                # print("ALL-TEST ACCURACY")
                test(round_num, devices[0], model_id, arch_testset, test_dataloader)
        
        # Validation
        if round_num not in DUPLICATE_MILESTONES:
            for device in round_devices:
                for model_id in model_id_lst:
                    if device.active[model_id] != 0:
                        validate(NUM_LOCAL_EPOCHS - 1, device, model_id)    # <- there are print statements here
                # Figure out rankings here
                device.update_ranking(prune)

        # Testing with IID data from device
        test_iid_results = []
        for index in range(len(devices)):
            device = devices[index]
            max_model = device.ranking.index(max(device.ranking))
            test_iid_results.append(float(test(round_num, device, max_model, device.device_testset, device.testloader)[1]))

        print(round_num, test_iid_results)
        tracker.update_per_device(test_iid_results)
        """
        active_arr_tracker = [sum(devices[i].active) for i in range(len(devices))]
        print(round_num, active_arr_tracker)
        for i in range(len(devices)):
            print(i, devices[i].ranking)
            print(i, " ", devices[i].ranking.index(max(devices[i].ranking)), " ", devices[i].ranking)
        """

        if algorithm == "FedCD" and round_num in DUPLICATE_MILESTONES:
        # duplicate all models
            # Run validation and update rankings for everyone
            for device in devices:
                for model_id in model_id_lst:
                    if device.active[model_id] != 0:
                        validate(NUM_LOCAL_EPOCHS - 1, device, model_id)    # <- there are print statements here
                # Figure out rankings here
                device.update_ranking(prune)

            # Number of nets to duplicate
            nets_to_create = len(model_id_lst)
            for model_id in range(nets_to_create):
                for device in devices:
                    # If model wasn't already removed
                    if device.active[model_id] != 0:    
                        device_net = ConvNet().cuda()
                        device_net.load_state_dict(device.nets[model_id]['net'].state_dict())
                        valid_loss_tracker = [copy.deepcopy(device.nets[model_id]['valid_loss_tracker'][-1])]
                        valid_acc_tracker = [100 - copy.deepcopy(device.nets[model_id]['valid_acc_tracker'][-1])]
                        optimizer = torch.optim.SGD(device_net.parameters(), lr=0.1, momentum=0.9,
                                                    weight_decay=5e-4)
                        rounds_passed = len(device.nets[model_id]['train_acc_tracker'])
                        scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer,
                                                                        milestones=[25-rounds_passed, 50-rounds_passed, 75-rounds_passed],
                                                                        gamma=0.1)
                        device.nets.append({
                            'net': device_net,
                            'optimizer': optimizer,
                            'scheduler': scheduler,
                            'train_loss_tracker': [],
                            'train_acc_tracker': [],
                            'valid_loss_tracker': valid_loss_tracker,
                            'valid_acc_tracker': valid_acc_tracker,
                            'test_loss_tracker': [],
                            'test_acc_tracker': [],
                        })
                        device.active.append(1)
                        # Heavily rank the devices that are underperforming for new models and vice versa
                        if len(valid_acc_tracker) > 0:
                            device.update_ranking(prune, removed = False, duplicate_model_id = model_id + nets_to_create, offset_rank = valid_acc_tracker[-1])
                        else:
                            device.update_ranking(prune)
                    # If model was already removed
                    else:                           
                        device.nets.append({
                            'valid_acc_tracker': [0.],
                        })
                        device.active.append(0)
                        device.update_ranking(prune, removed = True, duplicate_model_id = model_id + nets_to_create)

                model_id_lst.append(model_id + nets_to_create)
        # print('model id list:', model_id_lst)
        # print('best model:', [device.ranking.index(max(device.ranking)) for device in devices])
        # for device in devices:
            # print("device's active models:", device.active)
            # print("device " + str(device.idx) + " ranking: " + str(device.ranking))
            # print("archetype:", device.archetype)

        # TODO save both print after
	
    total_time = time.time() - start_time
    print('Total training time: {} seconds'.format(total_time))

---
###**Run Experiment**
---

In [11]:
import torchvision
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler

# load training and validation data from CIFAR-10
transform_train_valid = transforms.Compose([                                   
    transforms.RandomCrop(32, padding=4),                                       
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, 
                                        download=True,
                                        transform=transform_train_valid)
validset = torchvision.datasets.CIFAR10(root='./data', train=True, 
                                        download=True,
                                        transform=transform_train_valid)

# fraction of data reserved for validation
valid_fraction = 0.2
indices = list(range(len(trainset)))
split = int(np.floor(valid_fraction * len(trainset)))
    
np.random.shuffle(indices)
train_idx, valid_idx = indices[split:], indices[:split]
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, 
                                            sampler=valid_sampler, shuffle=False,
                                            num_workers=2)
validloader = torch.utils.data.DataLoader(validset, batch_size=128, 
                                            sampler=valid_sampler, shuffle=False,
                                            num_workers=2)
# load testing data
transform_test = transforms.Compose([                                           
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True,
                                       transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=128, shuffle=False,
                                         num_workers=2)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [12]:
# Train model on each device
# Get rankings on each device
# Update weights

net = ConvNet().cuda()
criterion = nn.CrossEntropyLoss()

devices_archetype = [[i//3] for i in range(NUM_DEVICES)]
devices_bias = [random.uniform(0.6, 0.7) for i in range(NUM_DEVICES)]

train_idxs = data_sampler(DISTRIBUTION, trainset, NUM_TRAIN_PER_DEVICE, devices_archetype, devices_bias)
valid_idxs = data_sampler(DISTRIBUTION, validset, NUM_VALID_PER_DEVICE, devices_archetype, devices_bias)
test_idxs_device = data_sampler(DISTRIBUTION, testset, NUM_TEST_PER_DEVICE, devices_archetype, devices_bias)

label_dict_test = gen_label_dict(testset)
test_idxs = [id for i in range(NUM_LABELS) for id in label_dict_test[i]]
random.shuffle(test_idxs)

arch_testset = DatasetSplit(testset, test_idxs)
test_dataloader = torch.utils.data.DataLoader(arch_testset, batch_size=128,
                                                shuffle=True, num_workers=2)

label_dict_valid = gen_label_dict(validset)

In [ ]:
# FedCD: Federated Cloning and Deletion
if RUN_FEDCD:
    federated_learning("FedCD")

# FedAvg: Federated Learning Baseline
if RUN_FEDAVG:
    federated_learning("FedAvg")


Devices 30
0 [8.0, 10.0, 9.6, 10.4, 7.6, 11.0, 8.6, 9.0, 9.4, 64.0, 68.8, 9.0, 7.4, 9.0, 7.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
1 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 62.0, 58.8, 69.0, 40.2, 9.0, 10.2, 10.2, 61.4, 68.0, 7.2, 13.2, 10.2, 8.0, 8.6, 14.2]
2 [15.2, 69.2, 13.2, 46.0, 47.4, 44.4, 28.8, 61.8, 55.8, 11.0, 9.2, 14.8, 11.4, 11.4, 12.6, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
3 [0.0, 62.0, 53.4, 58.8, 64.6, 55.4, 68.8, 70.2, 70.6, 0.0, 24.0, 0.0, 0.0, 0.0, 0.0, 61.6, 47.6, 57.6, 76.6, 75.4, 75.2, 45.6, 44.4, 41.0, 67.8, 66.8, 67.8, 41.6, 47.6, 53.2]
4 [0.0, 69.2, 66.0, 61.0, 65.2, 62.4, 62.4, 40.6, 53.4, 0.0, 61.6, 60.8, 0.0, 30.6, 0.0, 66.4, 54.8, 54.8, 83.6, 84.2, 75.2, 71.8, 69.8, 64.6, 81.4, 80.4, 77.8, 65.2, 68.2, 54.6]
5 [67.8, 69.0, 73.2, 76.0, 78.2, 74.8, 55.8, 33.8, 46.8, 12.2, 47.4, 55.2, 44.8, 45.4, 40.2, 61.8, 61.2, 68.8, 27.8, 39.8, 24.8, 37.4, 48.2, 0.0